In [1]:
install.packages('quantmod')

also installing the dependencies ‘xts’, ‘TTR’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library('quantmod')

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.


In [3]:
# returns monthly adjusted closing stock price data from a given list of stock symbols
monthly <- function (symbols) {
    # fetch stock prices from yahoo finance using quantmod
    fetch <- sapply(symbols, function(x) {
        getSymbols(x, src="yahoo", from="2000-01-01", to="2017-01-01", auto.assign=FALSE)
    })

    # convert the data to monthly
    data <- lapply(fetch, function(x) {
        to.monthly(x)
    })
    
    # extract the adjust closing price from the data
    adjusted <- lapply(data, function(x) {
        temp <- x[, 'x.Adjusted']
        names(temp) <- 'Adjusted'
        return(temp)
    })
    
    return(adjusted)
}

In [4]:
# stock symbols
symbols <- c('^hsi', '0005.hk', '0006.hk', '0322.hk', '0001.hk', '0293.hk', '0941.hk')

In [5]:
# create adjusted closing price zoo object
adjusted <- monthly(symbols)
df.adjusted <- do.call(cbind, adjusted)
names(df.adjusted) <- symbols

    As of 0.4-0, ‘getSymbols’ uses env=parent.frame() and
 auto.assign=TRUE by default.

 This  behavior  will be  phased out in 0.5-0  when the call  will
 default to use auto.assign=FALSE. getOption("getSymbols.env") and 
 getOptions("getSymbols.auto.assign") are now checked for alternate defaults

 This message is shown once per session and may be disabled by setting 
 options("getSymbols.warning4.0"=FALSE). See ?getSymbols for more details.


In [6]:
# create zoo object for storing stock returns
returns <- lapply(adjusted, function(x) {
    diff(x[, 'Adjusted']) / lag(x[, 'Adjusted'] , 1)
})

df.returns <- do.call(cbind, returns)
rsymbols <- sapply(symbols, function(x) {
    paste('r', x, sep='')
})
names(df.returns) <- rsymbols

In [7]:
# merge the adjusted stock prices with stock returns data
df <- merge(df.adjusted, df.returns)

In [8]:
# read base rate and compute the monthly risk free rate
base <- read.csv('base.csv')
df$base <- base[, 'base']
rf <- (base[, 'base']/100) / 12

names(df) <- c(symbols, rsymbols, 'base')

In [9]:
# compute the difference between risk free rate and stock returns

er <- apply(df.returns, 2, function(x) {
    x - rf
})

ersymbols <- sapply(symbols, function(x) {
    paste('er', x, sep='')
})

df.er <- zoo(er, order.by=index(df.returns))
names(df.er) <- ersymbols

In [10]:
# merge the excess / shortage in returns with the adjusted closing prices and stock returns zoo object
master <- merge(df, df.er)

In [11]:
# write the data frames for future use
write.zoo(master, 'data.csv', index.name='date', sep=',')
write.zoo(df.er, 'er.csv', index.name='date', sep=',')

In [12]:
# data frames for CAPM to run OLS
mtft <- df.er[-1, 1]
jtft <- df.er[-1, -1]